In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
#from nltk.corpus import stopwords
import nltk
import re
from tabulate import tabulate
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

2023-03-11 12:18:23.795793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 12:18:23.795843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import platform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os = platform.system()

match os.lower():
    case "darwin":
        path = "/Users/antoniobaio/Desktop/Progetti/ProgettiDS//BERT/IMDB.csv"
    case "linux":
        path = "/home/antonio/Scrivania/ProgettiDS/BERT/IMDB.csv"
    case "windows":
        path = "AGGIUNGI PATH"
        
print("Actual path: " + path + " and device = " + str(device))

Actual path: /home/antonio/Scrivania/ProgettiDS/BERT/IMDB.csv and device = cpu


In [3]:
df = pd.read_csv(path)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antonio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['review'] = df['review'].apply(lambda x: clean_text(x))
df['review'] = df['review'].str.replace(r'\bbr\b', '')

df

/tmp/ipykernel_8170/1984782482.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review'] = df['review'].str.replace(r'\bbr\b', '')


,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production filming techniqu...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive
...,...,...
49995,thought movie right good job creative original...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,catholic taught parochial elementary schools n...,negative
49998,going disagree previous comment side maltin on...,negative


In [6]:
def cat2num(value):
    if value=='positive': 
        return 1
    else: 
        return 0
    
df['sentiment'] = df['sentiment'].apply(cat2num)

df = df.rename(columns={'sentiment': 'label'})

df = df.sample(frac=1, random_state=42)

df

,review,label
33553,"really liked summerslam due look arena, curtai...",1
9427,many television shows appeal quite many differ...,1
199,film quickly gets major chase scene ever incre...,0
12447,jane austen would definitely approve one gwy...,1
39489,"expectations somewhat high went see movie, tho...",0
...,...,...
11284,shadow magic recaptures joy amazement first mo...,1
44732,found movie quite enjoyable fairly entertainin...,1
38158,avoid one terrible movie exciting pointless mu...,0
860,production quite surprise absolutely love obsc...,1


In [7]:
reviews = df.review.values
labels = df.label.values

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
input_ids = []
attention_mask = []
for rev in reviews:
    encoded_data = tokenizer.encode_plus(
    rev,
    add_special_tokens=True,
    max_length=512,
    pad_to_max_length = True,
    return_attention_mask= True,
    return_tensors='pt')
    input_ids.append(encoded_data['input_ids'])
    attention_mask.append(encoded_data['attention_mask'])
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/antonio/.pyenv/versions/3.10.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
i=random.randint(0,len(reviews)-1)
print('Original: ', list(reviews)[i])
print('Input IDs:', input_ids[i])

Original:  got stupid film ever seen spoilers ahead first all, plot stupid little kid weird move hotel father caretaker find kid gift, shining gift never ever anything anything except make kid seem cool movie gets boring boring man finally goes crazy goes rampage kill kid wife well, feels like else would sudden see naked woman tub man kisses realizes kissing dead corpse, utterly disgusting somehow black man enters hotel whacked axe kid woman take black man vehicle leave father, dies within minutes hypothermia movies complete waste time, falls right category music trashy, characters corny except jack nicholson, good actor , plot twisted fits description vomit, ending predictable, storyline slow, tedious, boring movie extremely overrated avoid movie costs surprised gotten high rating imdb
Input IDs: tensor([  101,  2288,  5236,  2143,  2412,  2464, 27594,  2545,  3805,  2034,
         2035,  1010,  5436,  5236,  2210,  4845,  6881,  2693,  3309,  2269,
        17600,  2424,  4845,  5592,

In [11]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(reviews) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(input_ids[index]))
  token_ids = [i.numpy() for i in input_ids[index]]
  attention = [i.numpy() for i in attention_mask[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(list(reviews)[index])
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

wow   picked local wal mart reading online released early following online time, buy film   wow   guess thing really struck editing, lack thereof time again, characters usually narrator whoever shown walkingand walkingand walking editor, know cut someone leaving point show arriving point b need show almost entire journey wow   actually ended feeling somewhat sorry actors involved seem given direction scenes look scared look happy, depending action added later date   wow   decided almost effects using cg beyond even ilm still employs miniatures sometimes one distracting uses green screen film constant rushing according end credits group people representing citizenry different towns cities, including london times folk coming going regard angle shot distance camera one shot london, appear least two men six feet tall walking behind narrator brother played star anthony piana without distracting mustache since gettysburg seen fake piece facial hair   wow   timothy hines talked film way beyon

In [12]:
dataset = TensorDataset(input_ids,attention_mask,labels)
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('Training Size - ',train_size)
print('Validation Size - ',val_size)

Training Size -  40000
Validation Size -  10000


In [13]:
train_dl = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),
                     batch_size = 16)
val_dl = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset),
                     batch_size = 36)

len(train_dl),len(val_dl)

(2500, 278)

In [14]:
model = BertForSequenceClassification.from_pretrained( 'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False)

model.to(device)
print(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

cpu


In [15]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [16]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps=1e-8)

/home/antonio/.pyenv/versions/3.10.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
from transformers import get_linear_schedule_with_warmup
epochs = 2
total_steps = len(train_dl)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [18]:
def accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat==label_flat)/len(label_flat)

In [19]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [],[]
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

In [20]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [21]:
from ipywidgets import FloatProgress
import tqdm
from tqdm import tqdm
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dl, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dl)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dl)
    val_acc = accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {val_acc}')


  0%|          | 0/2 [00:00<?, ?it/s]

: 

: 

In [ ]:
def Sentiment(sent):
    output_dir = './'
    tokenizer = BertTokenizer.from_pretrained(output_dir)
    model_loaded = BertForSequenceClassification.from_pretrained(output_dir)
    encoded_dict = tokenizer.encode_plus(
                        sent, 
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
        
    input_id = encoded_dict['input_ids']

    attention_mask = encoded_dict['attention_mask']
    input_id = torch.LongTensor(input_id)
    attention_mask = torch.LongTensor(attention_mask)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_loaded = model_loaded.to(device)
    input_id = input_id.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model_loaded(input_id, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs[0]
    index = logits.argmax()
    return index

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
output_dir = './'
tokenizer = BertTokenizer.from_pretrained(output_dir)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

## VECCHIO

In [9]:
index=0
print(' Original: ', reviews[index])

table = np.array([tokenizer.tokenize(reviews[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[index]))]).T

print(tabulate(table,headers = ['Tokens', 'Token IDs'],tablefmt = 'fancy_grid'))



 Original:  really liked summerslam due look arena, curtains look overall interesting reason anyways, could one best summerslam ever wwf lex luger main event yokozuna, time ok huge fat man vs strong man glad times changed terrible main event like every match luger terrible matches card razor ramon vs ted dibiase, steiner brothers vs heavenly bodies, shawn michaels vs curt hening, event shawn named big monster body guard diesel, irs vs kid, bret hart first takes doink takes jerry lawler stuff harts lawler always interesting, ludvig borga destroyed marty jannetty, undertaker took giant gonzalez another terrible match, smoking gunns tatanka took bam bam bigelow headshrinkers, yokozuna defended world title lex luger match boring terrible ending however deserves
╒═════════════╤═════════════╕
│ Tokens      │   Token IDs │
╞═════════════╪═════════════╡
│ really      │        2428 │
├─────────────┼─────────────┤
│ liked       │        4669 │
├─────────────┼─────────────┤
│ summers     │       

In [10]:
max_len = 0
l=[]
for rev in list(reviews):

    #Tokenizza il testo e aggiunge i tokens `[CLS]` e `[SEP]`
    input_ids = tokenizer.encode(rev, add_special_tokens=True)

    #Aggiorna la lunghezza massima delle frasi presenti nel dataset
    l.append(len(input_ids))
    max_len = max(max_len, len(input_ids))

if max_len>512:
  max_len=512
avg_len=int(sum(l)/len(l))

print('Max sentence length: ', max_len)
print('Avg sentence length: ', avg_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
input_ids = []
attention_masks = []

# For every tweet...
for rev in reviews:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        rev,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



/tmp/ipykernel_5907/3879750887.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [ ]:
#Stampa la prima frase come una lista di IDs
i=random.randint(0,len(reviews)-1)
print('Original: ', list(reviews)[i])
print('Input IDs:', input_ids[i])

Original:  hitting international acclaim silence lambs, director jonathan demme cut teeth making quirky comedies one like quite oscar winning american comedies could mention, fine concept, well paced, great performances, complicated romance simply funny pfeiffer mob widow moves city backwaters husband baldwin murdered crime boss killed stockwell takes fancy pfeiffer, wife reuhl furious complicate matters pfeiffer also falls cop trailing laugh minute pfeiffer, sporting hefty wig excellent widow, hyperactive ruehl modine good nice cop script simply devoid one liners, wit, humour punch lines verbal physical kind kind film demands result raises smiles best rather guffaws oozes charm, tediously short humour
Input IDs: tensor([  101,  7294,  2248, 10761,  4223, 12559,  2015,  1010,  2472,  5655,
        17183,  4168,  3013,  4091,  2437, 21864, 15952, 22092,  2028,  2066,
         3243,  7436,  3045,  2137, 22092,  2071,  5254,  1010,  2986,  4145,
         1010,  2092, 13823,  1010,  2307, 

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(reviews) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(input_ids[index]))
  token_ids = [i.numpy() for i in input_ids[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(reviews[index])
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

right, then, absolutely brilliant must intelligent quick understand humor covers attacks sorts topics, first moon landing, easter christmas, transvestitism, movies, herr doktor heimlich   averse swearing, us punctuate commas periods, uses f word also, laugh yourself, never watch feel fool   incidentally, watched stuff even saw perform live, far best work simply shines   might go far say glorious
╒═════════════╤═════════════╤══════════════════╕
│ Tokens      │   Token IDs │   Attention Mask │
╞═════════════╪═════════════╪══════════════════╡
│ [CLS]       │         101 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ right       │        2157 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ,           │        1010 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ then        │        2059 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ,           │        1010 │                1 │
├────────────

In [ ]:
#Creaun unico Dataset (di tensori) unendo i tensori relativi agli input, alle label e alle attention mask
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

#Divide randomicamente il dataset in training set e validation set
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

40,000 training samples
10,000 validation samples


In [ ]:
#Per eseguire il fine-tuning di BERT su un task specifico, è raccomandata una dimensione del batch pari a 16 o a 32
batch_size = 16

#Crea il DataLoaders per il training set e per il validation set
#Vengono presi gli elementoi del training set in ordine randomico
train_dataloader = DataLoader(
            train_dataset,  #Il training set
            sampler = RandomSampler(train_dataset), #Seleziona randomicamente i batch
            batch_size = batch_size #Esegue l'allenamento con questa dimensione dei batch
        )


validation_dataloader = DataLoader(
            val_dataset, #Il validation set
            sampler = SequentialSampler(val_dataset), #Seleziona i batch in maniera sequenziale
            batch_size = batch_size #Esegue una valutazione del modello con questa dimensione dei batch
        )

In [ ]:
# Viene caricato il modello pre-allenato BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #Usa il modello di BERT a 12-layer con un vocabolario tutto in lower case
    num_labels = 2, #Il numero delle label 
    output_attentions = False,
    output_hidden_states = False,
)

model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # learning_rate - quello standard è 5e-5
                  eps = 1e-8 # adam_epsilon  - quello standard è 1e-8.
                )

/home/antonio/.pyenv/versions/3.10.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 2

#Numero totale degli step di training, è dato da: [numero di batch] x [numero di epoche]
total_steps = len(train_dataloader) * epochs

#Viene creato lo scheduler del learning rate
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [55]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

#Misura il tempo totale che è stato necessario per il training
total_t0 = time.time()

for epoch_i in range(0, epochs):
    #               Training
    #Esegue un primo passo (completo) nel training set
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    #Misura quanto ci mette l'epoca in questione
    t0 = time.time()
    total_train_loss = 0

    #Mette il modello in training mode (presta attenzione al fatto che NON esegue l'allenamento del modello)
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        #Pulisce i gradienti precedentemente clacolati prima di eseguire un backward 
        #pass (PyTorch non lo fa di default perchè nell'allenamento di RNN è conveniente accumularli)
        model.zero_grad()
        #La documentazione relativa al modello la si può trovare al link sotto:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        #Ritorna un numero differente di parametri in base in base agli argomenti dati e alle flag settate.
        # Nel nostro caso ritorna la loss (perchè abbiamo dato un dataset etichettato in input)
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)  
        #Accumula la loss di allenamento su tutti i batch in modo da poter calcolare la perdita 
        #media alla fine. `loss` è un tensore contenente un singolo valore;                          
        loss = output.loss
        logits = output.logits
        total_train_loss += loss.item()
        #Esegue un backward pass per calcolare i gradienti
        loss.backward()
        #Ritaglia la norma dei gradienti a 1.0.
        #Questo per aiutare a prevenire il problema dei "gradienti che esplodono".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Aggiorna i parametri e fa uno step usando il gradiente calcolato.
        #L'ottimizzatore detta la "regola di aggiornamento": come i parametri 
        #vengono modificati in base ai loro gradienti, al tasso di apprendimento, ecc.
        optimizer.step()
        #Aggiorna il learning rate
        scheduler.step()

    #Calcola la perdita media di tutti i batch
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    #Misura quanto tempo è necessario per questa epoca
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    #               Validation
    #Dopo il completamento di ogni epoca di addestramento, misurariamo
    # le nostre prestazioni sul nostro set di convalida.
    print("")
    print("Running Validation...")

    t0 = time.time()

    #Mette il modello nel mode di valutazione

    model.eval()

    total_eval_accuracy = 0
    best_eval_accuracy = np.Inf
    total_eval_loss = 0
    nb_eval_steps = 0

    #Valuta i dati di una epoca
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #Dice a pytorch di non preoccuparsi di costruire il compute graph durante il passaggio 
        #in avanti, poiché questo è necessario solo per il backprop (fase di addestramento).
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()

        #Sposta i logits e le labels sulla CPU se stavamo usando la GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        #Calcola l'accuratezza per questo batch di test, e la accumula con quella di ogni batch
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    #Restituisce l'accuratezza
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    #Restituise la loss media
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    #Misura quanto tempo c'è voluto per la validazione
    validation_time = format_time(time.time() - t0)

    #Salva il modello migliore
    if avg_val_accuracy < best_eval_accuracy:
        torch.save(model, 'bert_model_bbc')
        best_eval_accuracy = avg_val_accuracy

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    #Memorizza tutte le statistiche per questa epoca
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

: 

: 